# Project 1






In [14]:
df = pd.read_csv(r"C:\Users\maxta\Downloads\chicago crime 2008-2012.csv")

In [27]:
df["Date"].str.replace("#", "")     #replace hashtags with regular string objects(easier to read) ** apprently need inplace 2 here

0          03/01/2008 12:00:00 AM
1          03/01/2008 12:00:00 AM
2          03/01/2008 12:00:00 AM
3          03/01/2008 12:00:00 AM
4          03/01/2008 12:00:00 AM
                    ...          
1530135    03/01/2012 12:00:00 AM
1530136    03/01/2012 12:00:00 AM
1530137    03/01/2012 12:00:00 AM
1530138    03/01/2012 12:00:00 AM
1530139    03/01/2012 12:00:00 AM
Name: Date, Length: 1530140, dtype: object

In [47]:
#df.drop('FBI Code',axis=1, inplace = True) #removing unneccessary columns (datacleaning)
#df.drop('Updated On',axis=1, inplace = True)

#missing_val = list(df.isna().sum())
#cols = list(df.columns)
#print(cols) 
#for i in range(len(cols)):
    #if (missing_val[i] == 0): 
        #cols[i]="Others" 
        #df = df.dropna() 
        #df.info()

#    Introduction

Chicago crimes 2008-2012: Do Financial downturns have an impact on Crime?
    
I will be investigating the impacts of crime by analyzing crime rates in different communities based on their relative poverty index.

In [57]:
#! pip install qeds
! pip install fiona 
#import geopandas as gpd
#import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib.ticker as ticker
#import numpy as np
#from matplotlib.pyplot import figure
#import seaborn as sea
#from datetime import datetime
#!pip install descartes
from shapely.geometry import Point

In [48]:
df2 = pd.read_csv(r"C:\Users\maxta\Downloads\below_poverty_level_by_community.csv")

#merging data together to observe area and community poverty with crime

In [56]:
df3 = pd.merge(df, df2, left_on=["Community Area"], right_on=["Community Area Number"], how='inner').sort_values(by=['PERCENT HOUSEHOLDS BELOW POVERTY'], ascending=True)
df3.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Location,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
1515239,8545393,HV220970,02/28/2012 12:01:00 AM,070XX N OSCEOLA AVE,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,RESIDENCE,False,False,...,"(42.008389253, -87.812274628)",9.0,Edison Park,1.1,3.3,6.5,7.4,35.3,40959,8.0
1514237,6867954,HR273380,04/16/2009 09:00:00 PM,072XX W IBSEN ST,0820,THEFT,$500 AND UNDER,RESIDENTIAL YARD (FRONT/BACK),False,False,...,"(42.006285351, -87.807655816)",9.0,Edison Park,1.1,3.3,6.5,7.4,35.3,40959,8.0
1514238,6867892,HR273235,04/17/2009 03:15:00 PM,072XX W DEVON AVE,0820,THEFT,$500 AND UNDER,FOREST PRESERVE,False,False,...,"(41.997764674, -87.807067431)",9.0,Edison Park,1.1,3.3,6.5,7.4,35.3,40959,8.0
1514239,6879266,HR284852,04/17/2009 05:00:00 PM,070XX N ORIOLE AVE,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,RESIDENCE,False,False,...,"(42.008364111, -87.816425274)",9.0,Edison Park,1.1,3.3,6.5,7.4,35.3,40959,8.0
1514240,6869019,HR273446,04/17/2009 05:52:00 PM,072XX W COYLE AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,True,True,...,"(42.007165159, -87.80742045)",9.0,Edison Park,1.1,3.3,6.5,7.4,35.3,40959,8.0


Begin Mapping crime with poverty index

In [102]:
communitydf = gpd.read_file(r"C:\Users\maxta\Downloads\Boundaries - Community Areas (current).zip")
#cleaning data
communitydf.drop('comarea',axis=1, inplace = True) #removing unneccessary columns (datacleaning)
communitydf.drop('comarea_id',axis=1, inplace = True)
communitydf.drop('area',axis=1, inplace = True)
communitydf.drop('area_num_1',axis=1, inplace = True)
communitydf.drop('perimeter',axis=1, inplace = True)
communitydf

,area_numbe,community,shape_area,shape_len,geometry
0,35,DOUGLAS,4.600462e+07,31027.054510,"POLYGON ((-87.60914 41.84469, -87.60915 41.844..."
1,36,OAKLAND,1.691396e+07,19565.506153,"POLYGON ((-87.59215 41.81693, -87.59231 41.816..."
2,37,FULLER PARK,1.991670e+07,25339.089750,"POLYGON ((-87.62880 41.80189, -87.62879 41.801..."
3,38,GRAND BOULEVARD,4.849250e+07,28196.837157,"POLYGON ((-87.60671 41.81681, -87.60670 41.816..."
4,39,KENWOOD,2.907174e+07,23325.167906,"POLYGON ((-87.59215 41.81693, -87.59215 41.816..."
...,...,...,...,...,...
72,74,MOUNT GREENWOOD,7.558429e+07,48665.130539,"POLYGON ((-87.69646 41.70714, -87.69644 41.706..."
73,75,MORGAN PARK,9.187734e+07,46396.419362,"POLYGON ((-87.64215 41.68508, -87.64249 41.685..."
74,76,OHARE,3.718356e+08,173625.984660,"MULTIPOLYGON (((-87.83658 41.98640, -87.83658 ..."
75,77,EDGEWATER,4.844999e+07,31004.830946,"POLYGON ((-87.65456 41.99817, -87.65456 41.998..."


In [104]:
communitydf.rename(columns = {'area_numbe':'Community Area Number', 'community':'COMMUNITY AREA NAME'}, inplace = True)
communitydf

,Community Area Number,COMMUNITY AREA NAME,shape_area,shape_len,geometry
0,35,DOUGLAS,4.600462e+07,31027.054510,"POLYGON ((-87.60914 41.84469, -87.60915 41.844..."
1,36,OAKLAND,1.691396e+07,19565.506153,"POLYGON ((-87.59215 41.81693, -87.59231 41.816..."
2,37,FULLER PARK,1.991670e+07,25339.089750,"POLYGON ((-87.62880 41.80189, -87.62879 41.801..."
3,38,GRAND BOULEVARD,4.849250e+07,28196.837157,"POLYGON ((-87.60671 41.81681, -87.60670 41.816..."
4,39,KENWOOD,2.907174e+07,23325.167906,"POLYGON ((-87.59215 41.81693, -87.59215 41.816..."
...,...,...,...,...,...
72,74,MOUNT GREENWOOD,7.558429e+07,48665.130539,"POLYGON ((-87.69646 41.70714, -87.69644 41.706..."
73,75,MORGAN PARK,9.187734e+07,46396.419362,"POLYGON ((-87.64215 41.68508, -87.64249 41.685..."
74,76,OHARE,3.718356e+08,173625.984660,"MULTIPOLYGON (((-87.83658 41.98640, -87.83658 ..."
75,77,EDGEWATER,4.844999e+07,31004.830946,"POLYGON ((-87.65456 41.99817, -87.65456 41.998..."


In [90]:
df3["Coordinates"] = list(zip(df3.Longitude, df3.Latitude))
df3["Coordinates"] = df3["Coordinates"].apply(Point)
df3.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,Coordinates
1515239,8545393,HV220970,02/28/2012 12:01:00 AM,070XX N OSCEOLA AVE,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,RESIDENCE,False,False,...,9.0,Edison Park,1.1,3.3,6.5,7.4,35.3,40959,8.0,POINT (-87.812274628 42.008389253)
1514237,6867954,HR273380,04/16/2009 09:00:00 PM,072XX W IBSEN ST,0820,THEFT,$500 AND UNDER,RESIDENTIAL YARD (FRONT/BACK),False,False,...,9.0,Edison Park,1.1,3.3,6.5,7.4,35.3,40959,8.0,POINT (-87.807655816 42.006285351)
1514238,6867892,HR273235,04/17/2009 03:15:00 PM,072XX W DEVON AVE,0820,THEFT,$500 AND UNDER,FOREST PRESERVE,False,False,...,9.0,Edison Park,1.1,3.3,6.5,7.4,35.3,40959,8.0,POINT (-87.807067431 41.997764674)
1514239,6879266,HR284852,04/17/2009 05:00:00 PM,070XX N ORIOLE AVE,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,RESIDENCE,False,False,...,9.0,Edison Park,1.1,3.3,6.5,7.4,35.3,40959,8.0,POINT (-87.816425274 42.008364111)
1514240,6869019,HR273446,04/17/2009 05:52:00 PM,072XX W COYLE AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,True,True,...,9.0,Edison Park,1.1,3.3,6.5,7.4,35.3,40959,8.0,POINT (-87.80742045 42.007165159)


In [113]:
df3["COMMUNITY AREA NAME"] = df3["COMMUNITY AREA NAME"].str.title()
df3["COMMUNITY AREA NAME"] = df3["COMMUNITY AREA NAME"].str.strip()
communitydf["COMMUNITY AREA NAME"] = communitydf["COMMUNITY AREA NAME"].str.title()
communitydf["COMMUNITY AREA NAME"] = communitydf["COMMUNITY AREA NAME"].str.strip()

In [120]:
chicago = communitydf.merge(df3, left_on= "COMMUNITY AREA NAME", right_on="COMMUNITY AREA NAME", how="inner")
chicago.drop('Location',axis=1, inplace = True)
chicago.drop('Longitude',axis=1, inplace = True)
chicago.drop('Latitude',axis=1, inplace = True)
chicago

,Community Area Number_x,COMMUNITY AREA NAME,shape_area,shape_len,geometry,ID,Case Number,Date,Block,IUCR,...,Year,Community Area Number_y,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,Coordinates
0,35,Douglas,4.600462e+07,31027.054510,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",6147368,HP238876,03/22/2008 03:00:00 PM,035XX S RHODES AVE,2820,...,2008,35.0,1.8,29.6,18.2,14.3,30.7,23791,47.0,POINT (-87.61448 41.83070)
1,35,Douglas,4.600462e+07,31027.054510,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",6150669,HP238707,03/22/2008 03:58:00 PM,005XX E BROWNING AVE,2027,...,2008,35.0,1.8,29.6,18.2,14.3,30.7,23791,47.0,POINT (-87.61229 41.82988)
2,35,Douglas,4.600462e+07,31027.054510,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",6147661,HP238728,03/22/2008 04:30:00 PM,006XX E WOODLAND PARK,0880,...,2008,35.0,1.8,29.6,18.2,14.3,30.7,23791,47.0,POINT (-87.61009 41.83210)
3,35,Douglas,4.600462e+07,31027.054510,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",6149124,HP240811,03/22/2008 05:00:00 PM,037XX S INDIANA AVE,1310,...,2008,35.0,1.8,29.6,18.2,14.3,30.7,23791,47.0,POINT (-87.62167 41.82685)
4,35,Douglas,4.600462e+07,31027.054510,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",6150144,HP238875,03/22/2008 05:49:00 PM,004XX E 35TH ST,1460,...,2008,35.0,1.8,29.6,18.2,14.3,30.7,23791,47.0,POINT (-87.61614 41.83117)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485225,9,Edison Park,3.163631e+07,25937.226841,"POLYGON ((-87.80676 42.00084, -87.80676 42.000...",7753233,HS556771,10/09/2010 11:00:00 PM,064XX N AVONDALE AVE,0820,...,2010,9.0,1.1,3.3,6.5,7.4,35.3,40959,8.0,POINT (-87.81076 41.99734)
1485226,9,Edison Park,3.163631e+07,25937.226841,"POLYGON ((-87.80676 42.00084, -87.80676 42.000...",7750716,HS559649,10/11/2010 03:00:00 PM,074XX N OSCEOLA AVE,1540,...,2010,9.0,1.1,3.3,6.5,7.4,35.3,40959,8.0,POINT (-87.81269 42.01568)
1485227,9,Edison Park,3.163631e+07,25937.226841,"POLYGON ((-87.80676 42.00084, -87.80676 42.000...",7758862,HS562876,10/12/2010 09:30:00 AM,074XX N OKETO AVE,0890,...,2010,9.0,1.1,3.3,6.5,7.4,35.3,40959,8.0,POINT (-87.81147 42.01592)
1485228,9,Edison Park,3.163631e+07,25937.226841,"POLYGON ((-87.80676 42.00084, -87.80676 42.000...",7746236,HS553893,10/07/2010 07:00:00 PM,067XX N OLMSTED AVE,0820,...,2010,9.0,1.1,3.3,6.5,7.4,35.3,40959,8.0,POINT (-87.81713 42.00229)


Creating variables for communities with 25 percent or (more/less) households living under the federal poverty level 

In [122]:
chicago["25% and above"] = chicago["PERCENT HOUSEHOLDS BELOW POVERTY"] >= 25.0
chicago["25% and below"] = chicago["PERCENT HOUSEHOLDS BELOW POVERTY"] <= 25.0
chicago.head()

,Community Area Number_x,COMMUNITY AREA NAME,shape_area,shape_len,geometry,ID,Case Number,Date,Block,IUCR,...,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,Coordinates,25% and above,25% and below
0,35,Douglas,4.600462e+07,31027.05451,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",6147368,HP238876,03/22/2008 03:00:00 PM,035XX S RHODES AVE,2820,...,1.8,29.6,18.2,14.3,30.7,23791,47.0,POINT (-87.61448 41.83070),True,False
1,35,Douglas,4.600462e+07,31027.05451,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",6150669,HP238707,03/22/2008 03:58:00 PM,005XX E BROWNING AVE,2027,...,1.8,29.6,18.2,14.3,30.7,23791,47.0,POINT (-87.61229 41.82988),True,False
2,35,Douglas,4.600462e+07,31027.05451,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",6147661,HP238728,03/22/2008 04:30:00 PM,006XX E WOODLAND PARK,0880,...,1.8,29.6,18.2,14.3,30.7,23791,47.0,POINT (-87.61009 41.83210),True,False
3,35,Douglas,4.600462e+07,31027.05451,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",6149124,HP240811,03/22/2008 05:00:00 PM,037XX S INDIANA AVE,1310,...,1.8,29.6,18.2,14.3,30.7,23791,47.0,POINT (-87.62167 41.82685),True,False
4,35,Douglas,4.600462e+07,31027.05451,"POLYGON ((-87.60914 41.84469, -87.60915 41.844...",6150144,HP238875,03/22/2008 05:49:00 PM,004XX E 35TH ST,1460,...,1.8,29.6,18.2,14.3,30.7,23791,47.0,POINT (-87.61614 41.83117),True,False
